<h1 align="center" , style="color:#B82727;"> Paradigmes d'apprentissage avancés appliqués à la détection des émotions</h1>

## **Importation des bibliothèques nécessaires**

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization,MaxPooling2D,Activation,Input

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import shutil
from sklearn.metrics import confusion_matrix , classification_report 
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.backend import clear_session
import random



## Parametres 

In [1]:
SEED = 12
IMG_HEIGHT = 48
IMG_WIDTH = 48
BATCH_SIZE = 64
NUM_CLASSES = 7
CLASS_LABELS  = ['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sadness', "Surprise"]
CLASS_LABELS_EMOJIS = ["👿", "🤢" , "😱" , "😊" , "😐 ", "😔" , "😲" ]

##  **Construction du modèle**

In [3]:
def cnn_model(num_classes):
    # Créer un modèle séquentiel
    model= tf.keras.models.Sequential()
    # Premier bloc de couches convolutives : 32 filtres
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(48, 48,1)))
    model.add(Conv2D(64,(3,3), padding='same', activation='relu' ))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    # Deuxième bloc : 128 filtres
    model.add(Conv2D(128,(5,5), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    # Troisième bloc : 512 filtres, augmentation de la régularisation  
    model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    # Répétition du troisième bloc pour une extraction de caractéristiques plus profonde
    model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    # Dernière répétition du troisième bloc
    model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    # Aplatir la sortie pour l'alimenter dans des couches denses
    model.add(Flatten()) 
    model.add(Dense(256,activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    # Couches denses pour la classification 
    model.add(Dense(512,activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    # Couche de sortie avec activation softmax pour la classification multiclasses
    model.add(Dense(256,activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    # Compiler le modèle avec l'optimiseur Adam et la perte d'entropie croisée catégorique
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(
    optimizer = Adam(learning_rate=0.0001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy'])
    return model

# Supervised learning

**Configurer trois générateurs pour préparer et optimiser les données d'entraînement, de validation et de test pour le modèle**

In [9]:
preprocess_fun = tf.keras.applications.densenet.preprocess_input
train_dir = '/kaggle/input/emotion-detection-fer/train'
test_dir = '/kaggle/input/emotion-detection-fer/test'

train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   rescale = 1./255,
                                   validation_split = 0.2
                                    
                                   
                                  
                                  )
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2
                                   
                                 )

train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                   target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = True , 
                                                    color_mode = "grayscale",
                                                    class_mode = "categorical",
                                                    subset = "training",
                                                    seed = 12
                                                   )

validation_generator = test_datagen.flow_from_directory(directory = train_dir,
                                                         target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                         batch_size = BATCH_SIZE,
                                                         shuffle  = True , 
                                                         color_mode = "grayscale",
                                                         class_mode = "categorical",
                                                         subset = "validation",
                                                         seed = 12
                                                        )

test_generator = test_datagen.flow_from_directory(directory = test_dir,
                                                   target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = False , 
                                                    color_mode = "grayscale",
                                                    class_mode = "categorical",
                                                    seed = 12
                                                  )

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [6]:
# liste de Callbacks  avec " Early Stopping" pour éviter l'overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=3)
callbacks_list = [early_stop]

In [12]:
model = cnn_model(7)

In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 48, 48, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 48, 48, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 24, 24, 128)    │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 12, 12, 512)    │       590,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 12, 12, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 6, 6, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 3, 3, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 5,940,743 (22.66 MB)

 Trainable params: 5,935,239 (22.64 MB)

 Non-trainable params: 5,504 (21.50 KB)

In [14]:
history = model.fit(x = train_generator, epochs=30, validation_data= validation_generator, callbacks = callbacks_list)

Epoch 1/30


2024-05-28 10:00:07.389898: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 1.16473, expected 0.928603
2024-05-28 10:00:07.389966: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 8: 1.1794, expected 0.943274
2024-05-28 10:00:07.389981: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 9: 0.892153, expected 0.656024
2024-05-28 10:00:07.389993: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 10: 1.163, expected 0.926874
2024-05-28 10:00:07.390010: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 11: 1.33797, expected 1.10184
2024-05-28 10:00:07.390024: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 13: 1.18042, expected 0.944295
2024-05-28 10:00:07.390035: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 19: 1.28869, expected 1.05256
2024-05-28 10:00:07.390045: E external/local_xla

  1/359 ━━━━━━━━━━━━━━━━━━━━ 3:04:42 31s/step - accuracy: 0.2656 - loss: 14.9324

I0000 00:00:1716890421.343833     236 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


273/359 ━━━━━━━━━━━━━━━━━━━━ 33s 392ms/step - accuracy: 0.1565 - loss: 14.8491

2024-05-28 10:02:15.472564: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 1.6176, expected 1.1528
2024-05-28 10:02:15.472632: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 2.31702, expected 1.85223
2024-05-28 10:02:15.472642: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 2.58123, expected 2.11644
2024-05-28 10:02:15.472650: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 2.73522, expected 2.27042
2024-05-28 10:02:15.472657: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 2.52884, expected 2.06404
2024-05-28 10:02:15.472665: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 2.80052, expected 2.33572
2024-05-28 10:02:15.472673: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 2.45587, expected 1.99107
2024-05-28 10:02:15.472681: E external/local_xla/xla/serv

358/359 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step - accuracy: 0.1593 - loss: 14.7715

2024-05-28 10:03:33.111100: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 1.55293, expected 1.11788
2024-05-28 10:03:33.111189: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 1.5362, expected 1.10116
2024-05-28 10:03:33.111207: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 2.00511, expected 1.57006
2024-05-28 10:03:33.111225: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 2.11493, expected 1.67989
2024-05-28 10:03:33.111238: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 1.78366, expected 1.34862
2024-05-28 10:03:33.111249: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 1.77945, expected 1.34441
2024-05-28 10:03:33.111260: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 1.60647, expected 1.17143
2024-05-28 10:03:33.111271: E external/local_xla/xla/ser

359/359 ━━━━━━━━━━━━━━━━━━━━ 224s 540ms/step - accuracy: 0.1593 - loss: 14.7698 - val_accuracy: 0.2050 - val_loss: 13.4801
Epoch 2/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 50s 138ms/step - accuracy: 0.1885 - loss: 13.5504 - val_accuracy: 0.2510 - val_loss: 12.3934
Epoch 3/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 49s 133ms/step - accuracy: 0.2036 - loss: 12.2913 - val_accuracy: 0.2487 - val_loss: 11.0531
Epoch 4/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 49s 134ms/step - accuracy: 0.2152 - loss: 10.9536 - val_accuracy: 0.2639 - val_loss: 9.7387
Epoch 5/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 49s 135ms/step - accuracy: 0.2231 - loss: 9.6075 - val_accuracy: 0.2749 - val_loss: 8.4733
Epoch 6/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 49s 135ms/step - accuracy: 0.2446 - loss: 8.3205 - val_accuracy: 0.2989 - val_loss: 7.2575
Epoch 7/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 49s 136ms/step - accuracy: 0.2647 - loss: 7.1181 - val_accuracy: 0.3008 - val_loss: 6.3186
Epoch 8/30
359/359 ━━━━━━━━━━━━━━━━━━━━ 49s 134ms/step - accuracy: 0.2990 - loss: 6.07

In [15]:
history = pd.DataFrame(history.history)

In [16]:
model.evaluate(test_generator)
preds = model.predict(test_generator)
y_preds = np.argmax(preds , axis = 1 )
y_test = np.array(test_generator.labels)

112/113 ━━━━━━━━━━━━━━━━━━━━ 0s 465ms/step - accuracy: 0.5419 - loss: 1.6948

2024-05-28 10:29:47.708050: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 1.6297, expected 1.1253
2024-05-28 10:29:47.708122: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 2.33636, expected 1.83196
2024-05-28 10:29:47.708132: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 2.37284, expected 1.86844
2024-05-28 10:29:47.708140: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 2.90532, expected 2.40092
2024-05-28 10:29:47.708148: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 2.27154, expected 1.76715
2024-05-28 10:29:47.708156: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 2.18271, expected 1.67831
2024-05-28 10:29:47.708175: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 2.44143, expected 1.93703
2024-05-28 10:29:47.708184: E external/local_xla/xla/serv

113/113 ━━━━━━━━━━━━━━━━━━━━ 53s 474ms/step - accuracy: 0.5427 - loss: 1.6929
113/113 ━━━━━━━━━━━━━━━━━━━━ 11s 91ms/step


In [17]:
data = pd.DataFrame({
    'y_true': y_test,
    'y_pred': y_preds
})
data.to_csv('data_evaluation.csv', index=False)
history.to_csv('training.csv', index=False)

# Self supervised learning

In [5]:
os.makedirs('/kaggle/working/train/1')
os.makedirs('/kaggle/working/train/0')
os.makedirs('/kaggle/working/test/0')
os.makedirs('/kaggle/working/test/1')
os.makedirs('/kaggle/working/train/2')

os.makedirs('/kaggle/working/test/2')

In [6]:
# Chemin du dossier source
source_folder = '/kaggle/input/imdb-wiki-faces-dataset/imdb_crop'

# Collecter les sous-dossiers du dossier source
subdirs = [os.path.join(source_folder, d) for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))]
subdirs = subdirs[:14]  
# Liste pour stocker les chemins complets des fichiers image
all_images = []

# Parcourir les sous-dossiers sélectionnés pour collecter tous les fichiers images
for subdir in subdirs:
    for file in os.listdir(subdir):
            all_images.append(os.path.join(subdir, file))

# Mélanger aléatoirement la liste des images
random.shuffle(all_images)

# Calculer l'index pour la séparation des 90% et 10%
split_index = int(0.9 * len(all_images))

# Copier les 80% des fichiers dans le premier dossier
for image_path in all_images[:split_index]:
    shutil.copy(image_path, '/kaggle/working/train/0')

# Copier les 20% restants dans le second dossier
for image_path in all_images[split_index:]:
    shutil.copy(image_path, '/kaggle/working/test/0')

print('Finished copying images.')

Finished copying images.


### **Tâche de prétexte** : Création de pseudo-étiquettes  

In [8]:
def rotate_and_copy_images(source_dir, target_dir_base):
    
    # Dossiers pour les images tournées
    dirs = {
        90: os.path.join(target_dir_base, '1'),
        180: os.path.join(target_dir_base, '2')
    }
    
   # Parcourir tous les fichiers dans le dossier source
    for filename in os.listdir(source_dir):
        filepath = os.path.join(source_dir, filename)
        if os.path.isfile(os.path.join(source_dir, filename)):
                image = Image.open(filepath)
                # Appliquer les rotations et sauvegarder dans les dossiers appropriés
                for angle in [90, 180]:
                    rotated_image = image.rotate(angle)
                    rotated_image.save(os.path.join(dirs[angle], filename))
            
        
        

# Utiliser la fonction
source_directory = '/kaggle/working/train/0'
destination_base_directory = '/kaggle/working/train'
rotate_and_copy_images(source_directory, destination_base_directory)

In [20]:
train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   rescale = 1./255,
                                   validation_split = 0.2,
                                   
                                  
                                  )
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2,
                            
                                 )
train_dir0 = '/kaggle/working/train'
test_dir0 = '/kaggle/working/test'
train_generator0 = train_datagen.flow_from_directory(directory = train_dir0,
                                                   target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = True , 
                                                    color_mode = "grayscale",
                                                    class_mode = "categorical",
                                                    subset = "training",
                                                    seed = 12
                                                   )

validation_generator0 = test_datagen.flow_from_directory(directory = train_dir0,
                                                         target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                         batch_size = BATCH_SIZE,
                                                         shuffle  = True , 
                                                         color_mode = "grayscale",
                                                         class_mode = "categorical",
                                                         subset = "validation",
                                                         seed = 12
                                                        )

test_generator0= test_datagen.flow_from_directory(directory = test_dir0,
                                                   target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = False , 
                                                    color_mode = "grayscale",
                                                    class_mode = "categorical",
                                                    seed = 12
                                                  )


Found 128868 images belonging to 3 classes.
Found 32214 images belonging to 3 classes.
Found 17901 images belonging to 3 classes.


In [55]:
clear_session()

In [24]:
model0 = cnn_model(3)

In [25]:
model0.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 48, 48, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 48, 48, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 24, 24, 128)    │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 12, 12, 512)    │       590,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 12, 12, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 6, 6, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 3, 3, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 5,939,715 (22.66 MB)

 Trainable params: 5,934,211 (22.64 MB)

 Non-trainable params: 5,504 (21.50 KB)

In [26]:
# Training the model on Pretext Task
history0 = model0.fit(x = train_generator0, epochs=10, validation_data= validation_generator0 , callbacks = callbacks_list)

Epoch 1/10
  90/2014 ━━━━━━━━━━━━━━━━━━━━ 2:51 89ms/step - accuracy: 0.3475 - loss: 13.8787

2024-05-28 10:34:09.867125: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 1.27494, expected 0.700202
2024-05-28 10:34:09.867202: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 2.61358, expected 2.03885
2024-05-28 10:34:09.867211: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 2.446, expected 1.87126
2024-05-28 10:34:09.867219: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 2.33526, expected 1.76052
2024-05-28 10:34:09.867227: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 2.62103, expected 2.0463
2024-05-28 10:34:09.867234: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 2.59956, expected 2.02483
2024-05-28 10:34:09.867242: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 2.02822, expected 1.45348
2024-05-28 10:34:09.867250: E external/local_xla/xla/serv

2013/2014 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.5440 - loss: 12.0013

2024-05-28 10:37:54.337044: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 1.21004, expected 0.77311
2024-05-28 10:37:54.337106: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 1.8404, expected 1.40347
2024-05-28 10:37:54.337115: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 1.93134, expected 1.4944
2024-05-28 10:37:54.337123: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 2.20933, expected 1.77239
2024-05-28 10:37:54.337131: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 1.72546, expected 1.28853
2024-05-28 10:37:54.337139: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 1.94324, expected 1.50631
2024-05-28 10:37:54.337146: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 1.67036, expected 1.23343
2024-05-28 10:37:54.337154: E external/local_xla/xla/serv

2014/2014 ━━━━━━━━━━━━━━━━━━━━ 262s 120ms/step - accuracy: 0.5441 - loss: 11.9994 - val_accuracy: 0.9137 - val_loss: 6.1550
Epoch 2/10
2014/2014 ━━━━━━━━━━━━━━━━━━━━ 217s 107ms/step - accuracy: 0.8827 - loss: 4.8953 - val_accuracy: 0.9735 - val_loss: 1.7617
Epoch 3/10
2014/2014 ━━━━━━━━━━━━━━━━━━━━ 226s 112ms/step - accuracy: 0.9494 - loss: 1.3915 - val_accuracy: 0.9845 - val_loss: 0.5416
Epoch 4/10
2014/2014 ━━━━━━━━━━━━━━━━━━━━ 213s 105ms/step - accuracy: 0.9662 - loss: 0.5172 - val_accuracy: 0.9811 - val_loss: 0.3496
Epoch 5/10
2014/2014 ━━━━━━━━━━━━━━━━━━━━ 215s 106ms/step - accuracy: 0.9717 - loss: 0.3602 - val_accuracy: 0.9916 - val_loss: 0.2685
Epoch 6/10
2014/2014 ━━━━━━━━━━━━━━━━━━━━ 227s 112ms/step - accuracy: 0.9755 - loss: 0.3163 - val_accuracy: 0.9915 - val_loss: 0.2533
Epoch 7/10
2014/2014 ━━━━━━━━━━━━━━━━━━━━ 212s 105ms/step - accuracy: 0.9767 - loss: 0.2925 - val_accuracy: 0.9802 - val_loss: 0.2648
Epoch 8/10
2014/2014 ━━━━━━━━━━━━━━━━━━━━ 220s 109ms/step - accuracy: 0.

In [27]:
model0.save("model_ssl.h5")
model0.evaluate(test_generator0)
preds0 = model0.predict(test_generator0)
y_preds0 = np.argmax(preds0 , axis = 1 )
y_test0 = np.array(test_generator0.labels)

280/280 ━━━━━━━━━━━━━━━━━━━━ 21s 75ms/step - accuracy: 0.9826 - loss: 0.2545
280/280 ━━━━━━━━━━━━━━━━━━━━ 20s 68ms/step


In [28]:
data0 = pd.DataFrame({
    'y_true': y_test0,
    'y_pred': y_preds0
})
data0.to_csv('data_evaluation0.csv', index=False)

### **Tâche en aval**

In [110]:
clear_session()
model0 = load_model('model_ssl.h5')

In [111]:
model0.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 48, 48, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 48, 48, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 24, 24, 128)    │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 12, 12, 512)    │       590,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 12, 12, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 6, 6, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 3, 3, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 5,939,717 (22.66 MB)

 Trainable params: 5,934,211 (22.64 MB)

 Non-trainable params: 5,504 (21.50 KB)

 Optimizer params: 2 (12.00 B)

In [112]:
model0.pop()
model0.add(keras.layers.Dense(7, name='dense_3', activation='softmax'))

### **Entrainement du modèle**

les poids des couches convolutives seront figés et seules les couches denses seront entraînées

In [113]:
# Freezing the Convolutional Layers while keeping Dense layers as Trainable
for layer in model0.layers:
    if layer.name in ['dense_3', 'dense', 'batch_normalization_5', 'dense_1','dense_2','batch_normalization_6','batch_normalization_7','dropout_5', 'dropout_6', 'dropout_7']:
        layer.trainable=True
    else:
        layer.trainable=False
      

In [115]:
model0.compile(optimizer = Adam(learning_rate=0.001),loss='categorical_crossentropy', metrics=["accuracy"])


In [116]:
model0.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 48, 48, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 48, 48, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 48, 48, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 24, 24, 128)    │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 12, 12, 512)    │       590,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 12, 12, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 6, 6, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 6, 6, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 3, 3, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 5,940,743 (22.66 MB)

 Trainable params: 398,087 (1.52 MB)

 Non-trainable params: 5,542,656 (21.14 MB)

In [117]:
history2 = model0.fit(x = train_generator, epochs=7, validation_data= validation_generator, callbacks = callbacks_list)

Epoch 1/7
359/359 ━━━━━━━━━━━━━━━━━━━━ 100s 230ms/step - accuracy: 0.1918 - loss: 2.1962 - val_accuracy: 0.2649 - val_loss: 1.8214
Epoch 2/7
359/359 ━━━━━━━━━━━━━━━━━━━━ 48s 132ms/step - accuracy: 0.2594 - loss: 1.8699 - val_accuracy: 0.3050 - val_loss: 1.7521
Epoch 3/7
359/359 ━━━━━━━━━━━━━━━━━━━━ 47s 129ms/step - accuracy: 0.2741 - loss: 1.7998 - val_accuracy: 0.3221 - val_loss: 1.7160


### **Fine-tuning**

Utilisation d'un taux d'apprentissage (learning rate) réduit pour réaliser des ajustements légers des poids du modèle. 

In [118]:
for layer in model0.layers:
        layer.trainable=True

In [119]:
model0.compile(optimizer = Adam(learning_rate=0.0001),loss='categorical_crossentropy', metrics=["accuracy"])

In [120]:
history2 = model0.fit(x = train_generator, epochs=40, validation_data= validation_generator)

Epoch 1/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 83s 173ms/step - accuracy: 0.3998 - loss: 1.7802 - val_accuracy: 0.5093 - val_loss: 1.5927
Epoch 2/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 48s 131ms/step - accuracy: 0.5110 - loss: 1.5956 - val_accuracy: 0.5600 - val_loss: 1.5054
Epoch 3/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 47s 129ms/step - accuracy: 0.5426 - loss: 1.5730 - val_accuracy: 0.5692 - val_loss: 1.5263
Epoch 4/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 48s 130ms/step - accuracy: 0.5507 - loss: 1.5623 - val_accuracy: 0.5217 - val_loss: 1.6323
Epoch 5/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 48s 130ms/step - accuracy: 0.5635 - loss: 1.5634 - val_accuracy: 0.5750 - val_loss: 1.5192
Epoch 6/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 84s 136ms/step - accuracy: 0.5772 - loss: 1.5509 - val_accuracy: 0.5933 - val_loss: 1.5089
Epoch 7/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 48s 132ms/step - accuracy: 0.5827 - loss: 1.5483 - val_accuracy: 0.5837 - val_loss: 1.5501
Epoch 8/40
359/359 ━━━━━━━━━━━━━━━━━━━━ 47s 129ms/step - accuracy: 0.5895 - loss: 1

In [121]:
model0.evaluate(test_generator)
preds2 = model0.predict(test_generator)
y_preds2 = np.argmax(preds2 , axis = 1 )
y_test = np.array(test_generator.labels)

113/113 ━━━━━━━━━━━━━━━━━━━━ 16s 145ms/step - accuracy: 0.6163 - loss: 1.5983
113/113 ━━━━━━━━━━━━━━━━━━━━ 11s 91ms/step


In [122]:
history2 = pd.DataFrame(history2.history)
data2 = pd.DataFrame({
    'y_true': y_test,
    'y_pred': y_preds2
})
data2.to_csv('data_evaluation2.csv', index=False)
history2.to_csv('training2.csv', index=False)